Imports and function to get the population of the departments

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from unidecode import unidecode
import geopandas


def crawlpop():
    wikiurl = 'https://en.wikipedia.org/wiki/List_of_Colombian_departments_by_population'
    table_class="wikitable sortable jquery-tablesorter"
    response=requests.get(wikiurl)
    #print(response.status_code)
    soup = BeautifulSoup(response.text, 'html.parser')
    coltable=soup.find('table',{'class':"wikitable"})
    df=pd.read_html(str(coltable))
    df=pd.DataFrame(df[0])
    #print(df.head())
    dfpop = pd.DataFrame(columns=["dep","population"])
    dfpop["dep"] = df["Department"]
    dfpop["population"] = df["Population (2020)[1]"]
    #print(dfpop)
    dfpop.to_csv("data/temppop.csv")

crawlpop()

Preparing the data

In [2]:
df = pd.read_csv("data/DV.csv", low_memory= False)
#taken from https://www.kaggle.com/datasets/oscardavidperilla/domestic-violence-in-colombia
df2 = pd.read_csv("data/temppop.csv", encoding="utf-8")
df = df[df.GENERO != "NO REPORTA"]
df = df[df.GENERO != "-"]
df = df[df.MUNICIPIO != "NO REPORTA"]
df = df[df["ARMAS MEDIOS"] != "NO REPORTA"]
df = df[df["ARMAS MEDIOS"] != "NO REPORTADO"]
df = df[df["ARMAS MEDIOS"] != "-"]
df = df.dropna()
df["DEPARTAMENTO"] = df["DEPARTAMENTO"].apply(lambda x: unidecode(x))
df.loc[df["MUNICIPIO"] == "BOGOTÁ D.C. (CT)", "DEPARTAMENTO"] = "SANTAFE DE BOGOTA D.C"


deps = []
#deps = [['ATLÁNTICO', 'BOYACÁ', 'CAQUETÁ', 'CASANARE', 'CUNDINAMARCA', 'SUCRE', 'VALLE', 'HUILA', 'ANTIOQUIA', 'ARAUCA', 'BOLÍVAR', 'CALDAS', 'CAUCA', 'CESAR', 'CHOCÓ', 'CÓRDOBA', 'MAGDALENA', 'META', 'NARIÑO', 'NORTE DE SANTANDER', 'PUTUMAYO', 'RISARALDA', 'SANTANDER', 'TOLIMA', 'VAUPÉS', 'GUAVIARE', 'GUAJIRA', 'QUINDÍO', 'AMAZONAS', 'VICHADA', 'GUAINÍA', 'SAN ANDRÉS', 'NO REPORTA']]
for x in df["DEPARTAMENTO"]:
   if x not in deps:
        deps.append(x)
  
df2.dep = df2.dep.str.upper()

#df2["population"] = df2["population"].replace([df2[df2["dep"] == "CUNDINAMARCA"]["population"]], df2[df2["dep"] == "CUNDINAMARCA"]["population"].values + df2[df2["dep"] == "BOGOTÁ"]["population"].values)
#df2 = df2.drop(df2[df2["dep"] == "BOGOTÁ"].index)
df2["dep"] = df2["dep"].replace("BOGOTÁ", "SANTAFE DE BOGOTA D.C")
df2["dep"] = df2["dep"].replace("LA GUAJIRA", "GUAJIRA")
df2["dep"] = df2["dep"].replace("SAN ANDRÉS Y PROVIDENCIA", "SAN ANDRES")
df2["dep"] = df2["dep"].replace("VALLE DEL CAUCA", "VALLE")
df2["dep"] = df2["dep"].apply(lambda x: unidecode(x))

In [3]:
df

,DEPARTAMENTO,MUNICIPIO,CODIGO DANE,ARMAS MEDIOS,FECHA HECHO,GENERO,GRUPO ETARIO,CANTIDAD
0,ATLANTICO,BARRANQUILLA (CT),8001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
1,BOYACA,DUITAMA,15238000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
2,CAQUETA,PUERTO RICO,18592000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,MASCULINO,ADULTOS,1
3,CASANARE,MANÍ,85139000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
4,SANTAFE DE BOGOTA D.C,BOGOTÁ D.C. (CT),11001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,1
...,...,...,...,...,...,...,...,...
476963,VICHADA,PUERTO CARREÑO (CT),99001000,SIN EMPLEO DE ARMAS,44335,MASCULINO,ADULTOS,1
476964,VICHADA,SANTA ROSALÍA,99624000,SIN EMPLEO DE ARMAS,44199,FEMENINO,ADULTOS,1
476965,VICHADA,SANTA ROSALÍA,99624000,SIN EMPLEO DE ARMAS,44250,FEMENINO,MENORES,1
476966,VICHADA,SANTA ROSALÍA,99624000,SIN EMPLEO DE ARMAS,44262,FEMENINO,ADULTOS,1


Create a list that for each department divides the set of cases into different factors

In [4]:
a = []

for x in df:
    if x == "ARMAS MEDIOS" or x == "GENERO" or x == "GRUPO ETARIO":
        for y in df[x].unique():
            dftemp = df[df[x] == y]
            dftemp = dftemp["DEPARTAMENTO"].value_counts().to_dict()
            match x:
                case "ARMAS MEDIOS":
                    z = "Weapon_of_crime"
                    match y:
                        case "ARMA BLANCA / CORTOPUNZANTE":
                            z2 = "stabbing_weapon"
                            
                        case "ARMA DE FUEGO":
                            z2 = "firearm"
                            
                        case "CONTUNDENTES":
                            z2 = "blunt_weapon"
                            
                        case "CORTANTES":
                            z2 = "slashing_weapon"
                            
                        case "CORTOPUNZANTES":
                            z2 = "sharp_weapon2"
                            
                        case "PUNZANTES":
                            z2 = "sharp_weapon"
                            
                        case "SIN EMPLEO DE ARMAS":
                            z2 = "no_weapon"
                            
                        case "ESCOPOLAMINA":
                            z2 = "narcotics" 
                              
                case "GENERO":
                    z = "gender"
                    match  y:
                        case "MASCULINO":
                            z2 = "male"
                            
                        case "FEMENINO":
                            z2 = "female"
                              
                case "GRUPO ETARIO":
                    z = "agegroup"
                    match y:
                        case "ADULTOS":
                            z2 = "adult"
                            
                        case "ADOLESCENTES":
                            z2 = "teenager"
                             
                        case "MENORES":
                            z2 = "minor"
                            

            #for key in dftemp.keys():
                #dftemp[key] = (dftemp[key], z, z2)
            dftemp = [z, z2, dftemp]
        #dftemp.append(y)
            a.append(dftemp)


Check if the Data is correct

In [5]:
dep1 = sorted(deps)
dep2 = df2.sort_values("dep").reset_index()["dep"]

depvergleich = pd.DataFrame(columns=["dep1" , "dep2"])

depvergleich["dep1"] = dep1
depvergleich["dep2"] = dep2

for i in range(len(dep1)):
    if dep1[i] != dep2[i]:
        print(i, dep1[i], dep2[i])

writes data into df.csv

In [33]:
df["ARMAS MEDIOS"] = df["ARMAS MEDIOS"].replace("ARMA BLANCA / CORTOPUNZANTE","stabbing weapon")
df["ARMAS MEDIOS"] = df["ARMAS MEDIOS"].replace("ARMA DE FUEGO","firearm")
df["ARMAS MEDIOS"] = df["ARMAS MEDIOS"].replace("CONTUNDENTES","blunt weapon")
df["ARMAS MEDIOS"] = df["ARMAS MEDIOS"].replace("CORTANTES","slashing weapon")
df["ARMAS MEDIOS"] = df["ARMAS MEDIOS"].replace("CORTOPUNZANTES","stabbing weapon")
df["ARMAS MEDIOS"] = df["ARMAS MEDIOS"].replace("PUNZANTES","stabbing weapon")
df["ARMAS MEDIOS"] = df["ARMAS MEDIOS"].replace("SIN EMPLEO DE ARMAS","no weapon")
df["ARMAS MEDIOS"] = df["ARMAS MEDIOS"].replace("ESCOPOLAMINA","narcotics")
df["GENERO"] = df["GENERO"].replace("MASCULINO","male") 
df["GENERO"] = df["GENERO"].replace("FEMENINO","female")
df["GRUPO ETARIO"] = df["GRUPO ETARIO"].replace("ADULTOS","adult")
df["GRUPO ETARIO"] = df["GRUPO ETARIO"].replace("ADOLESCENTES","teenager")
df["GRUPO ETARIO"] = df["GRUPO ETARIO"].replace("MENORES","minor") 
df['MUNICIPIO'] = df['MUNICIPIO'].str.replace(r' \(CT\)$', '', regex=True)
df['MUNICIPIO'] = df['MUNICIPIO'].apply(lambda x: unidecode(x))
df["MUNICIPIO"] = df['MUNICIPIO'].replace("BOGOTA D.C.","BOGOTA")
df["MUNICIPIO"] = df['MUNICIPIO'].replace("SAN ANDRES DE TUMACO","TUMACO")
df["MUNICIPIO"] = df['MUNICIPIO'].replace("EL CARMEN DE VIBORAL","CARMEN DE VIBORAL")
df["MUNICIPIO"] = df['MUNICIPIO'].replace("SAN ANDRES SOTAVENTO","SAN ANDRES DE SOTAVENTO")
df["MUNICIPIO"] = df['MUNICIPIO'].replace("MARIQUITA","SAN SEBASTIAN DE MARIQUITA")

df.to_csv("data/dv_data.csv")

In [56]:
citydata = pd.read_csv("data/worldcities.csv")
citydata = citydata[citydata["country"] == "Colombia"]
citydata["city_ascii"] = citydata["city_ascii"].apply(lambda x: x.upper())
citydata["admin_name"] = citydata["admin_name"].apply(lambda x: unidecode(x).upper())
citydata["admin_name"] = citydata["admin_name"].replace("VALLE DEL CAUCA","VALLE")
citydata["admin_name"] = citydata["admin_name"].replace("LA GUAJIRA","GUAJIRA")
citydata["admin_name"] = citydata["admin_name"].replace("SAN ANDRES Y PROVIDENCIA","SAN ANDRES")
citydata["admin_name"] = citydata["admin_name"].replace("SANTAFE","SANTAFE DE BOGOTA D.C")
citydata.to_csv("data/worldcities.csv")

In [68]:
import json

citydata = pd.read_csv("data/worldcities.csv")
df = pd.read_csv("data/dv_data.csv")
counts = df["MUNICIPIO"].value_counts()
#counts.to_csv("data/counts.csv")

result = {}
for city in citydata["city_ascii"]:
    try:
        depa = str(citydata[citydata["city_ascii"] == city].admin_name).split()[1]
        if depa == "SANTAFE":
            depa = "SANTAFE DE BOGOTA D.C"

        if depa not in result:
            result[depa] = []
        
        result[depa].append({
            "city": city,
            "long": float(citydata[citydata["city_ascii"] == city].lng),
            "lat": float(citydata[citydata["city_ascii"] == city].lat),
            "cases": counts[city]
        })
    except (KeyError, TypeError):
        pass
    

#with open("data/counts.json", 'w') as file:
    #file.write(f'{result}')


C:\Users\slomo\AppData\Local\Temp\ipykernel_15252\3855688598.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "long": float(citydata[citydata["city_ascii"] == city].lng),
C:\Users\slomo\AppData\Local\Temp\ipykernel_15252\3855688598.py:21: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  "lat": float(citydata[citydata["city_ascii"] == city].lat),


In [70]:
#json.dumps(result).replace("'", '"')
import numpy as np

def convert_int64(item):
    if isinstance(item, np.int64):
        return int(item)
    return item

updated_data = json.dumps(result, default=convert_int64)
print(result)
updated_data

with open("data/counts.json", 'w') as file:
    file.write(f'{updated_data}')

{'SANTAFE DE BOGOTA D.C': [{'city': 'BOGOTA', 'long': -74.0722, 'lat': 4.7111, 'cases': 27302}], 'CAUCA': [{'city': 'TIMBIO', 'long': -76.6839, 'lat': 2.3445, 'cases': 267}, {'city': 'POPAYAN', 'long': -76.6092, 'lat': 2.4542, 'cases': 6426}, {'city': 'SANTANDER DE QUILICHAO', 'long': -76.4833, 'lat': 3.0167, 'cases': 1445}, {'city': 'PUERTO TEJADA', 'long': -76.4167, 'lat': 3.25, 'cases': 452}, {'city': 'PIENDAMO', 'long': -76.5284, 'lat': 2.6408, 'cases': 386}, {'city': 'MIRANDA', 'long': -76.25, 'lat': 3.25, 'cases': 168}, {'city': 'CAJIBIO', 'long': -76.6333, 'lat': 2.6333, 'cases': 210}, {'city': 'PATIA', 'long': -77.0833, 'lat': 2.1667, 'cases': 142}, {'city': 'BUENOS AIRES', 'long': -76.6667, 'lat': 2.9167, 'cases': 98}, {'city': 'CALDONO', 'long': -76.5333, 'lat': 2.8, 'cases': 97}, {'city': 'CORINTO', 'long': -76.2594, 'lat': 3.1739, 'cases': 170}, {'city': 'SILVIA', 'long': -76.3833, 'lat': 2.6167, 'cases': 136}, {'city': 'INZA', 'long': -76.0667, 'lat': 2.55, 'cases': 132}, 

In [73]:
json.loads(updated_data)

{'SANTAFE DE BOGOTA D.C': [{'city': 'BOGOTA',
   'long': -74.0722,
   'lat': 4.7111,
   'cases': 27302}],
 'CAUCA': [{'city': 'TIMBIO', 'long': -76.6839, 'lat': 2.3445, 'cases': 267},
  {'city': 'POPAYAN', 'long': -76.6092, 'lat': 2.4542, 'cases': 6426},
  {'city': 'SANTANDER DE QUILICHAO',
   'long': -76.4833,
   'lat': 3.0167,
   'cases': 1445},
  {'city': 'PUERTO TEJADA', 'long': -76.4167, 'lat': 3.25, 'cases': 452},
  {'city': 'PIENDAMO', 'long': -76.5284, 'lat': 2.6408, 'cases': 386},
  {'city': 'MIRANDA', 'long': -76.25, 'lat': 3.25, 'cases': 168},
  {'city': 'CAJIBIO', 'long': -76.6333, 'lat': 2.6333, 'cases': 210},
  {'city': 'PATIA', 'long': -77.0833, 'lat': 2.1667, 'cases': 142},
  {'city': 'BUENOS AIRES', 'long': -76.6667, 'lat': 2.9167, 'cases': 98},
  {'city': 'CALDONO', 'long': -76.5333, 'lat': 2.8, 'cases': 97},
  {'city': 'CORINTO', 'long': -76.2594, 'lat': 3.1739, 'cases': 170},
  {'city': 'SILVIA', 'long': -76.3833, 'lat': 2.6167, 'cases': 136},
  {'city': 'INZA', 'lo

Creates dataframe with the number of cases as well as the population of each department, as well as a normalized number of cases (cases / (population/1000)). And writes the Dataframe into data.csv

In [8]:

#b = df.groupby("DEPARTAMENTO").count()
dft = df["DEPARTAMENTO"].value_counts().to_dict().items()

data = pd.DataFrame(columns = ["dep", "cases", "population"])

dep = []
cases = []
population = []
norm = []

for x,y in dft:
    depart = unidecode(x)
    dep.append(depart)
    cases.append(y)
    p = int(df2[df2["dep"] == x].population)
    population.append(p)
    norm.append(round((y/(p/1000)), 2))

for i in range(len(a)):
    sublist = []
    var1 = a[i][0]
    var2 = a[i][1]
    for x,y in dft:
        if x in a[i][2]:
            varcas = a[i][2][x]
        else:
            varcas = 0
        sublist.append(varcas)
    #would write all combinations of the chosen factors from a into the dataframe
    #data["{}_{}".format(var1, var2)] = sublist


data["dep"] = dep
data["cases"] = cases
data["population"] = population
#data["norm"] = norm

data.to_csv("data/popdata.csv")

C:\Users\slomo\AppData\Local\Temp\ipykernel_15252\3785566858.py:15: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  p = int(df2[df2["dep"] == x].population)


Write all the factors into the geojson as properties (not needed on the current version)

In [9]:
with open("data/col.geojson") as file:
    dfgeo = geopandas.read_file(file)

#norm2 = []
cases2 = []
population2 = []
for index, row in dfgeo.iterrows():
    if row["NOMBRE_DPT"] in dep:
        #n = float(data[data["dep"] == row["NOMBRE_DPT"]].norm)
        c = int(data[data["dep"] == row["NOMBRE_DPT"]].cases)  
        p = int(data[data["dep"] == row["NOMBRE_DPT"]].population)
    else:
        n = 0.0
        c = 0
        p = 0
    #norm2.append(n)
    cases2.append(c)
    population2.append(p)

#dfgeo["norm"] = norm2
dfgeo["cases"] = cases2
dfgeo["population"] = population2
for i in range(len(a)):
    sublist = []
    var1 = a[i][0]
    var2 = a[i][1]
    for index, row in dfgeo.iterrows():
        if row["NOMBRE_DPT"] in a[i][2]:
            varcas = a[i][2][row["NOMBRE_DPT"]]
        else:
            varcas = 0
        sublist.append(varcas)
    dfgeo["{}__{}".format(var1, var2)] = sublist

#dfgeo.to_file("data/col_with_attributes.geojson", driver='GeoJSON')

C:\Users\slomo\AppData\Local\Temp\ipykernel_15252\3261683577.py:10: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  c = int(data[data["dep"] == row["NOMBRE_DPT"]].cases)
C:\Users\slomo\AppData\Local\Temp\ipykernel_15252\3261683577.py:11: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  p = int(data[data["dep"] == row["NOMBRE_DPT"]].population)
